Image processing
-----------------
first , we need to import some packages.

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

Color
--------------
Notice color Sequentially 

* opencv    : (B,G,R)

* matplolib : (R,G,B)

Now we use matplotlib show these images.

Usually , we use black-white images to do something , because it can reduce the complexity. 

In [ ]:
coin_path = "data/image_processing/coin.png"
image = cv2.imread(coin_path)
image_plt = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
plt.figure()
plt.subplot(1,2,1)
plt.imshow(image_plt)
## reference : https://www.cnblogs.com/denny402/p/5122594.html
plt.subplot(1,2,2)
plt.imshow(gray, cmap=plt.get_cmap('gray'))

Blurry
------------

If there are a lot of noise in image , we couldn't distinguish what the object is. 

So we have to use a function to reduce noice. One way is using blurring.

Next , we'll use Gaussian Blur to make images blurry.

* Gaussian api on opencv : cv2.GaussianBlur( image , ( w , h ) , std ) 

In [ ]:
blurred = {
            1 :  cv2.GaussianBlur(gray, (1, 1),0),
            3 :  cv2.GaussianBlur(gray, (3, 3),0),
            5 :  cv2.GaussianBlur(gray, (5, 5),0),
            7 :  cv2.GaussianBlur(gray, (7, 7),0),
            9 :  cv2.GaussianBlur(gray, (9, 9),0)    
          } 
blurred_index = 9 
plt.figure()
plt.imshow(blurred[blurred_index], cmap=plt.get_cmap('gray'))

edge
-------

we can use ``canny`` to find the edge.
* cv2.canny(image, upper_threshold , lower_threshold )

In [ ]:
fig = plt.figure()
for index in range(1,10,2):
    canny = cv2.Canny(blurred[index], 30, 150)
    ax = fig.add_subplot(330 + index)
    ax.set_title(index)
    plt.subplot(3,3,index)
    plt.imshow(canny , cmap=plt.get_cmap('gray'))

Contour
--------------

Notce : 

When use 

* cv2.drawContours()original 

image will be modified.

In [ ]:
fig = plt.figure()
coin_canny = cv2.Canny(blurred[9], 30, 150)
# reference : https://blog.csdn.net/on2way/article/details/46812121
ret, binary = cv2.threshold(coin_canny,127,255,cv2.THRESH_BINARY)

# reference : https://blog.csdn.net/hjxu2016/article/details/77833336
contours,hierarchy= cv2.findContours(binary, cv2.RETR_TREE , cv2.CHAIN_APPROX_SIMPLE) #cv2.RETR_TREE
image = cv2.imread(coin_path)
coin_contours_image = cv2.drawContours(image,contours,-1,(0,0,255),thickness=0)
coin_contours_plt = cv2.cvtColor(coin_contours_image, cv2.COLOR_BGR2RGB)
plt.imshow(coin_contours_plt)

we also can find the center. Use :

* cv2.moments(contours)

Marks :

* cv2.circle(image, (cX,cY), radius, color, thickness)

In [ ]:
# reference : https://chtseng.wordpress.com/2016/12/05/opencv-contour%E8%BC%AA%E5%BB%93/
clone_center = coin_contours_image.copy()
for cnt in contours:
    Moments = cv2.moments(cnt)
    cX = int(Moments['m10'] / Moments['m00'])
    cY = int(Moments['m01'] / Moments['m00'])
    clone_center = cv2.circle(clone_center,(cX,cY),10,(1,227,254),-1)
    
    
clone_center = cv2.cvtColor(clone_center, cv2.COLOR_BGR2RGB)
plt.figure()
plt.imshow(clone_center)

Use fitting rectangle

We can use :

* fit rectangle : cv2.boundingRect(Contours)

In [ ]:
image = cv2.imread(coin_path)

for cnt in contours: 
    (x,y,w,h) = cv2.boundingRect(cnt)
    cv2.rectangle(image, (x, y), (x + w, y + h), (0, 255, 0), 2)

image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) 
plt.imshow(image)


Mask
----------

First , we can find the contours 

In [ ]:
mask_path  = "data/image_processing/mask.png"
mask_image = cv2.imread(mask_path)
clone_mask_image = mask_image.copy()
gray = cv2.cvtColor(clone_mask_image, cv2.COLOR_BGR2GRAY)
(contours, _) = cv2.findContours(gray.copy(), cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)
clone_mask_image = cv2.drawContours(clone_mask_image, contours, -1, (0, 255, 0), 5)
clone_mask_image = cv2.cvtColor(clone_mask_image, cv2.COLOR_BGR2RGB)
plt.figure()
plt.imshow(mask_image)
plt.figure()
plt.imshow(clone_mask_image)

Next , accorinding to the contour , we can make mask.

In [ ]:
for cnt in contours:        
    mask = np.zeros(gray.shape, dtype="uint8")
    mask = cv2.drawContours(mask, [cnt] , -1 , 255, -1) #255  →白色, -1→塗滿
    new_image = cv2.bitwise_and(mask_image,mask_image,mask=mask)
    new_image = cv2.cvtColor(new_image, cv2.COLOR_BGR2RGB)
    plt.figure()
    plt.imshow(new_image)